In [9]:
import numpy as np
import pandas as pd

In [10]:
from typing import List, Dict

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
songs = pd.read_csv('songdata.csv')

In [14]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [15]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [16]:
songs['text'] = songs['text'].str.replace(r'\n', '')

In [17]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [18]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [19]:
cosine_similarities = cosine_similarity(lyrics_matrix)

In [20]:
similarities = {}

In [21]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [22]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [23]:
recommedations = ContentBasedRecommender(similarities)

In [24]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [25]:
recommedations.recommend(recommendation)

The 4 recommended songs for Driftin' Blues are:
Number 1:
Keep On Keepin' On by Allman Brothers Band with 0.293 similarity score
--------------------
Number 2:
Ship Of Fools by Doors with 0.241 similarity score
--------------------
Number 3:
Lonely Sea by Beach Boys with 0.233 similarity score
--------------------
Number 4:
Homeland Africa by Boney M. with 0.206 similarity score
--------------------


In [32]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [33]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Keep Falling In Love are:
Number 1:
All I Can Do by Dolly Parton with 0.281 similarity score
--------------------
Number 2:
Love To Love by The Monkees with 0.277 similarity score
--------------------
Number 3:
Love To Love by Neil Diamond with 0.262 similarity score
--------------------
Number 4:
That's How Strong My Love Is by Otis Redding with 0.253 similarity score
--------------------
